In [1]:
import numba
import numpy as np
from numba import cuda

#init
M = 1024
K = 1024
N = 1024
a = np.arange(M * K).reshape(M, K).astype(np.float64)
b = np.ones(K * N).reshape(K, N).astype(np.float64)
c = np.zeros(M * N).reshape(M, N).astype(np.float64)


#cuda allocation
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)
#d_c = cuda.device_array((dim, dim), dtype=np.float32)

In [3]:
%timeit c = a @ b #1024 1024 1024 -> 17.2microsec, -> int 9.11ms !!

5.24 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
# verify result

def verify(a, b, c):
  cpu_res = np.dot(a, b)
  np.testing.assert_almost_equal(c, cpu_res, decimal=2, )

In [5]:
@cuda.jit
def matrix_mul(a, b, c, M, K, N):
  i, j = cuda.grid(2)
  if i < M and j < N:
    c[i][j] = 0.0
    for k in range(K):
      c[i][j] += a[i][k] * b[k][j]

threads_per_block = (32, 32)
blocks = (32, 32)
matrix_mul[blocks, threads_per_block](d_a, d_b, d_c, M, K, N)
host_c = d_c.copy_to_host()
host_c

verify(a, b, host_c)

In [6]:
%timeit matrix_mul[blocks, threads_per_block](d_a, d_b, d_c, M, K, N); cuda.synchronize()

93.2 ms ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
#changing i, j to be j, i for Coalesced Access
#using tmp as a local variable
#to improve performance

@cuda.jit
def matrix_mul(a, b, c, M, K, N):
  j, i = cuda.grid(2)
  if i < M and j < N:
    tmp = 0.0
    for k in range(K):
      tmp += a[i][k] * b[k][j]
    c[i][j] = tmp

threads_per_block = (32, 32)
blocks = (32, 32)
%timeit matrix_mul[blocks, threads_per_block](d_a, d_b, d_c, M, K, N); cuda.synchronize()
host_c = d_c.copy_to_host()
verify(a, b, host_c)

10.7 ms ± 6.42 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
#stride for large matrix up to dim = 32 * 32 * 32 * 32 * 32 * 32
@cuda.jit
def matrix_mul(a, b, c, M, K, N):
  y, x = cuda.grid(2)
  strideX, strideY = cuda.gridsize(2)

  for i in range(x, M, strideX):
    for j in range(y, N, strideY):
      tmp = 0.0
      for k in range(K):
        tmp += a[i][k] * b[k][j]
      c[i][j] = tmp

threads_per_block = (32, 32)
blocks = (32, 32)
%timeit matrix_mul[blocks, threads_per_block](d_a, d_b, d_c, M, K, N); cuda.synchronize()
host_c = d_c.copy_to_host()
verify(a, b, host_c)

13.2 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from numba import cuda, float32

TPB = 32

@cuda.jit
def matrix_mul(A, B, C, dim):
  col, row = cuda.grid(2)
  x = cuda.threadIdx.x
  y = cuda.threadIdx.y
  
  sa = cuda.shared.array(shape=(32, 32), dtype=float32)
  sb = cuda.shared.array(shape=(32, 32), dtype=float32)
  
  tmp = 0.0
  for tile in range(0, cuda.gridDim.x):
    j = tile * 32 + x
    i = tile * 32 + y

    # Coalesced access
    sa[y][x] = A[row][j]
    sb[y][x] = B[i][col]

    cuda.syncthreads()

    for k in range(0, 32):
      tmp += sa[y][k] * sb[k][x] #no bank conflicts

    cuda.syncthreads()
  
  C[row][col] = tmp

threads_per_block = (TPB, TPB)
blocks = (32, 32)
%timeit matrix_mul[blocks, threads_per_block](d_a, d_b, d_c, 2048); cuda.synchronize()
host_c = d_c.copy_to_host()
#verify(a, b, host_c)


16.4 ms ± 61.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
from numba import cuda, float32

TPB = 32

@cuda.jit
def matrix_mul(A, B, C, dim):
  thx, thy = cuda.grid(2)
  strideX, strideY = cuda.gridsize(2)

  sa = cuda.shared.array(shape=(32, 32), dtype=float32)
  sb = cuda.shared.array(shape=(32, 32), dtype=float32)

  for col in range(thx, dim, strideX):
    for row in range(thy, dim, strideY):
      #col, row = cuda.grid(2)
      
      x = cuda.threadIdx.x
      y = cuda.threadIdx.y
      
      tmp = 0.0
      
      for tile in range(0, cuda.gridDim.x * 2):
        j = tile * 32 + x
        i = tile * 32 + y

        # Coalesced access
        sa[y][x] = A[row][j]
        sb[y][x] = B[i][col]

        cuda.syncthreads()

        for k in range(0, 32):
          tmp += sa[y][k] * sb[k][x] #no bank conflicts

        cuda.syncthreads()
      
      C[row][col] = tmp

threads_per_block = (TPB, TPB)
blocks = (32, 32)
%timeit matrix_mul[blocks, threads_per_block](d_a, d_b, d_c, 2048); cuda.synchronize()
host_c = d_c.copy_to_host()
verify(a, b, host_c)


CudaAPIError: [700] Call to cuCtxSynchronize results in UNKNOWN_CUDA_ERROR

In [ ]:
#!make attention for float32 precision because large number can accumulate error always make test using double precesion

#some number of blocks per grid even if they are small like 64 result of UNKNOWENE CUDA ERROR,
# if that happen the execution of the next kernel will fail even if the kernel is valid

#testing the matrix multiolication using shared memory in numba using GTX950M of my laptop result of slitly slow compared of
# the just using local variable, but using c/c++ cuda programming language the result was very fast (x7.5) from 100ms to 13ms
# the reason i don't know

# using stride of 32 in matrix multiplication is conveinte because it can traite matrix of large number up to 1 billion 
  # 32 * 32 * 32 * 32 * 32 *32 and the memory of the gpu is limited it is of the order gibyte

# there is some advance teqnics to get even fast kernel but i need more time to invastagate them 
  # https://github.com/siboehm/SGEMM_CUDA/tree/master

# using pytorch a python libarary for training ai models the matrix multiplication is so fast using gpu 2ms

# in numpy matrix multiplication using int32 is to slow compared to using float32 (1s vs 9ms)

